In [1]:
import os
import pandas as pd
import random
import subprocess
import numpy as np

In [2]:
def write_ins(file_name,b,h):
    with open(file_name,"w") as f:
        f.write("h="+str(h) + ";\r\n")
        f.write("w=" + str(h)+ ";\r\n")
        f.write("b = [|")
        for i in range(0,int(len(b)/ h)):
            f.write(str(b[i*h:(i+1)*h]).replace("[","").replace("]","|"))
        f.write("]")

In [3]:
def write_input(file_name,h):
    with open(file_name,"w") as f:
        f.write("h="+str(h) + ";\r\n")
        f.write("w=" + str(h)+ ";\r\n")

In [4]:
def get_instance(cmd):
    gen_p = subprocess.Popen(cmd,stdout=subprocess.PIPE,shell=True) 
    gen_out = gen_p.stdout.readlines()
    for i in range(0,len(gen_out)):
        gen_out[i] = gen_out[i].decode("utf-8")
    gen_out = ''.join(gen_out)
    gen_out = gen_out.split("\r\n----------\r\n")
    instance = gen_out[0].split("\r\n")
    return instance

In [5]:
def get_ori(instanc,dim):
    instance = np.array(instanc[0].split(" "))
    instance = [ int(i) for i in instance[0:len(instance)-1] ]
    result = instance[0:dim*dim]
    b = instance[dim*dim:dim*dim*2]
    return result,b

In [14]:
def gen_ori(dim):
    result = np.random.randint(0,2,dim*dim)
    board = np.zeros(dim*dim)
    for i in range(0,len(result)):
        if result[i] == 1:
            board[i] = -1
        else:
            num = 0
            if i - 1 - dim in range(0,dim*dim) and (i%dim !=0):
                if result[i - 1 - dim] == 1:
                    num = num + 1
            if i - dim in range(0,dim*dim):
                if result[i - dim] == 1:
                    num = num + 1
            if i - dim +1 in range(0,dim*dim) and (i+1)%dim !=0:
                if result[i - dim +1] == 1:
                    num = num + 1
            if i - 1 in range(0,dim*dim) and (i%dim) !=0:
                if result[i - 1] == 1:
                    num = num + 1
            if i +1 in range(0,dim*dim) and (i+1)%dim !=0:
                if result[i +1] == 1:
                    num = num + 1
            if i + dim -1 in range(0,dim*dim) and (i%dim !=0):
                if result[i + dim -1] == 1:
                    num = num + 1
            if i + dim in range(0,dim*dim):
                if result[i + dim] == 1:
                    num = num + 1
            if i + dim + 1 in range(0,dim*dim) and (i+1)%dim !=0:
                if result[i + dim +1] == 1:
                    num = num + 1
            board[i] = num
    return result,board

In [18]:
t1,t2 = gen_ori(3)

In [19]:
t1

array([0, 0, 1, 0, 1, 0, 0, 1, 0])

In [20]:
t2

array([ 1.,  2., -1.,  2., -1.,  3.,  2., -1.,  2.])

In [39]:
def gen_start(b):
    result =[]
    num = 0
    index = np.arange(0,len(b))
    rep = random.sample(list(index),int(len(index)/2))
    for i in range(0,len(b)):
        if i in rep:
            result.append(-1)
        else:
            result.append(int(b[i]))
    return result

In [40]:
def modify(start,target):
    index = []
    for i in range(0,len(start)):
        if start[i] != target[i]:
            index.append(i)
    change_index = np.random.choice(index)
    start[change_index] = int(target[change_index])
    return start

In [ ]:
ins_num = 0
dim = 15
puzzle = r"C:\Users\Brainless\ResearchProject\Minizinc\Minesweeper\minesweeper-gen.mzn"
input = r"C:\Users\Brainless\ResearchProject\Minizinc\Minesweeper\input.dzn"
solver = r"C:\Users\Brainless\ResearchProject\Minizinc\Minesweeper\minesweeper.mzn"
write_input(input,dim)
while ins_num<=10:
    seed = random.randint(0,10000000)
    cmd = 'minizinc ' + "-r "+ str(seed) + r"  --solver Gecode " +puzzle + " " + input
    instance = get_instance(cmd)
    out =""
    while len(out) != 2:
        result,target = get_ori(instance,dim)
        path = r"C:\Users\Brainless\ResearchProject\Minizinc\Minesweeper\instancetest.dzn"
        write_ins(path,target,dim)
        cmd_solver = 'minizinc ' + "-r "+ str(seed) + r" -n 2 --solver Gecode " + solver + " " + path
        p = subprocess.Popen(cmd_solver,stdout=subprocess.PIPE,shell=True) 
        out = p.stdout.readlines()
        for j in range(0,len(out)):
            out[j] = out[j].decode("utf-8")
        out = ''.join(out)
        out = out.split("\r\n----------\r\n")  
    b = gen_start(target)
    path = r"C:\Users\Brainless\ResearchProject\Minizinc\Minesweeper\instance"+str(ins_num)+".dzn"
    write_ins(path,b,dim)
    cmd_solver = 'minizinc ' + "-r "+ str(seed) + r" -n 2 --solver Gecode " + solver + " " + path
    p = subprocess.Popen(cmd_solver,stdout=subprocess.PIPE,shell=True) 
    out = p.stdout.readlines()
    for j in range(0,len(out)):
        out[j] = out[j].decode("utf-8")
    out = ''.join(out)
    out = out.split("\r\n----------\r\n")    
    while len(out) != 2:
        b = modify(b,target)
        path = r"C:\Users\Brainless\ResearchProject\Minizinc\Minesweeper\instance"+str(ins_num)+".dzn"
        write_ins(path,b,dim)
        cmd_solver = 'minizinc ' + "-r "+ str(seed) + r" -n 2 --time-limit 1000 --solver Gecode " + solver + " " + path
        p = subprocess.Popen(cmd_solver,stdout=subprocess.PIPE,shell=True) 
        out = p.stdout.readlines()
        for j in range(0,len(out)):
            out[j] = out[j].decode("utf-8")
        out = ''.join(out)
        out = out.split("\r\n----------\r\n")
    ins_num = ins_num+1

In [59]:
target

[]

In [41]:
ins_num = 0
dim = 7
puzzle = r"C:\Users\Brainless\ResearchProject\Minizinc\Minesweeper\minesweeper-gen.mzn"
input = r"C:\Users\Brainless\ResearchProject\Minizinc\Minesweeper\input.dzn"
solver = r"C:\Users\Brainless\ResearchProject\Minizinc\Minesweeper\minesweeper.mzn"
write_input(input,dim)
seed = np.random.randint(10000)
while ins_num<=10:
    a,target = gen_ori(dim)
    b= gen_start(target)
    path = r"C:\Users\Brainless\ResearchProject\Minizinc\Minesweeper\instance"+str(ins_num)+".dzn"
    write_ins(path,b,dim)
    cmd_solver = 'minizinc ' + "-r "+ str(seed) + r" -n 2 --time-limit 1000 --solver Gecode " + solver + " " + path
    p = subprocess.Popen(cmd_solver,stdout=subprocess.PIPE,shell=True) 
    out = p.stdout.readlines()
    for j in range(0,len(out)):
        out[j] = out[j].decode("utf-8")
    out = ''.join(out)
    out = out.split("\r\n----------\r\n")    
    while len(out) != 2:
        b = modify(b,target)
        path = r"C:\Users\Brainless\ResearchProject\Minizinc\Minesweeper\instance"+str(ins_num)+".dzn"
        write_ins(path,b,dim)
        cmd_solver = 'minizinc ' + "-r "+ str(seed) + r" -n 2 --time-limit 1000 --solver Gecode " + solver + " " + path
        p = subprocess.Popen(cmd_solver,stdout=subprocess.PIPE,shell=True) 
        out = p.stdout.readlines()
        for j in range(0,len(out)):
            out[j] = out[j].decode("utf-8")
        out = ''.join(out)
        out = out.split("\r\n----------\r\n")
    ins_num = ins_num+1

ValueError: a must be non-empty